In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset, load_from_disk
import torch
import random
print(torch.cuda.is_bf16_supported()) 

True


# Load all resources

## Load the model

In [2]:
model_name = "TinyLlama/TinyLlama_v1.1"

tokenizer = AutoTokenizer.from_pretrained(
    model_name
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16
)

tokenizer.save_pretrained("tinyllama_bf16")
model.save_pretrained("tinyllama_bf16")

## Load the datasets

In [ ]:
gutenberg = load_dataset("manu/project_gutenberg", split="fr", streaming=False)
gutenberg.save_to_disk("gutenberg")

In [ ]:
gallica = load_dataset("PleIAs/French-PD-Books", split = "train", streaming=False)
gallica.save_to_disk("gallica")

# Test the base model

In [11]:
tokenizer = AutoTokenizer.from_pretrained(
    "./tinyllama_bf16"
)
model = AutoModelForCausalLM.from_pretrained(
    "./tinyllama_bf16",
    torch_dtype=torch.bfloat16
).to("cuda")

In [12]:
prompt = "K. ouvrit la porte. "

for i in range(10):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=1000,
            temperature=0.8,
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.1
        )
    
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"== generation {i+1} ==")
    print(generated)
    print("\n")

== generation 1 ==
K. ouvrit la porte. Ça était fait pour que vous me voyez, m'écouter et me prendre à l'épreuve.

— Oui, mais n'oubliez pas qu'on n'est pas vraiment amis, vous et moi.

— Mais je suis certain que nous sommes de bons amis.

Elle se met à rire d'un air délicieux, comme si elle ne savait plus d'où venait cette joie. Il lui arrivait parfois de rêver à elle quand ils étaient tous deux dans les coulisses de l'opéra. C'était comme si on était en pleine ville, un endroit calme, où le monde entier se confondait. Pourtant, ils se trouvaient toujours à un niveau différent : ce qui passait entre eux au théâtre était l'illusion de la séparation qui laissait la rue aux passants ; puisqu'elle avait une carrière, elle pouvait faire des rencontres avec les autres, les autres artistes, même sans avoir jamais été à l'Opéra. Quand elles s'apercevaient ensemble, elles étaient plus ouvertes. Elle ne voulait pas leur manquer ; il suffisait d'appeler à la fin du concert. Comme souvent, elle é

## Check Gallica content

In [15]:
gallica = load_from_disk("gallica")
ntexts = len(gallica)
print(ntexts)

text_lengths = [len(gallica[i]["complete_text"]) for i in range(ntexts)]
min_len = min(text_lengths)
max_len = max(text_lengths)
avg_len = sum(text_lengths) / len(text_lengths)
print(f"Gallica corpus: {ntexts} texts with an average of {avg_len} character per text (min: {min_len}, max: {max_len})")

sample_size = 1000
i = 0
for i in range(10):
    item = gallica[random.randint(0, ntexts - 1)]
    text = item["complete_text"]

    max_start = len(text) - sample_size
    char_start = random.randint(0, max_start)
    substring = text[char_start:char_start + sample_size]
    print(f"== content {i+1} ==")
    print(substring)
    print("\n")

Loading dataset from disk:   0%|          | 0/204 [00:00<?, ?it/s]

289577
Gallica corpus: 289577 texts with an average of 340506.4779316037 character per text (min: 1, max: 30471621)
== content 1 ==
Français. » 
Ainsi parloit le bonhomme Voltaire , 
 
Et tous en choeur les diables de fa coût 
Applaudiffoient en hurlant à Pentour. 
Sur son brasier aboyant à son tour ,. 
Machiavel le força de se taire ,. 
Et de rentrer en son obscur séjour. 
« Oui, disoit-il, dans ce fameux ouvrage 
» Qui m'a valu ces charbons dévorans , 
» J'ouvris jadis une école aux tyrans , 
» Que j'enivrois de mon humeur sauvage : 
» Mais par prudence , aux peuples gémiffans , 
» Je n'en donnois qu'un seul, selon l'usage ; 
» Et vos décrets , auguste Aréopage , 
» Aux bons Français en donnent plusieurs cents. » 
Plus loin , au fond d'une grotte enflammée » Une voix douce, &amp; pourtant animée Par les soupirs , les sanglots , les hoquets , Disoit : H Pourquoi suis-je ici consumée ? » Que ne vivois-je au tems de ces décrets !.. ; w D'un sot époux à triple rang de cornes , W On redou

## Check Gutenberg content

In [18]:
gutenberg = load_from_disk("gutenberg")
ntexts = len(gutenberg)
print(ntexts)

text_lengths = [len(gutenberg[i]["text"]) for i in range(ntexts)]
min_len = min(text_lengths)
max_len = max(text_lengths)
avg_len = sum(text_lengths) / len(text_lengths)
print(f"Gutenberg FR corpus: {ntexts} texts with an average of {avg_len} character per text (min: {min_len}, max: {max_len})")

sample_size = 1000
i = 0
for i in range(10):
    item = gutenberg[random.randint(0, ntexts - 1)]
    text = item["text"]

    max_start = len(text) - sample_size
    char_start = random.randint(0, max_start)
    substring = text[char_start:char_start + sample_size]
    print(f"== content {i+1} ==")
    print(substring)
    print("\n")

5493
Gutenberg FR corpus: 5493 texts with an average of 414738.24776988896 character per text (min: 23460, max: 3167656)
== content 1 ==
rés par le mganga.

«Les Ouanyamouézi ont peu de formalités civiles ou religieuses. Quand
une femme est sur le point d'accoucher, elle se retire dans les
jungles, et revient au bout de quelques heures avec son enfant sur le
dos, et souvent une charge de bois sur la tête. Lorsque la couche est
double, ce qui heureusement est plus rare que chez les Cafres, l'un
des jumeaux est tué, et la mère emmaillotte une gourde qu'elle met
dormir avec le survivant. Si l'épouse meurt sans postérité, le veuf
réclame à son beau-père la somme qu'il avait donnée pour l'avoir; si
elle laisse un enfant, celui-ci hérite de la somme.

«La naissance, toutes les fois que les parents en ont le moyen, est
célébrée par une orgie; du reste, pas de cérémonies baptismales. Les
enfants appartiennent au père, qui a sur eux un droit absolu, et peut
les tuer ou les vendre sans encourir 

# Test the Gallica, French literacy-trained model

In [9]:
final_path = "./gallica_tinyllama_bf16"
model = AutoModelForCausalLM.from_pretrained(
    final_path,
    torch_dtype=torch.bfloat16,
    device_map="cuda"
)
tokenizer = AutoTokenizer.from_pretrained(final_path)

In [ ]:
prompt = "K. ouvrit la porte. "

for i in range(10):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=1000,
            temperature=0.8,
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.1
        )
    
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"== generation {i} ==")
    print(generated)
    print("\n")

# Test the final, Kafka fine-tuned model

In [ ]:
final_path = "./kafka_tinyllama_bf16"
model = AutoModelForCausalLM.from_pretrained(
    final_path,
    torch_dtype=torch.bfloat16,
    device_map="cuda"
)
tokenizer = AutoTokenizer.from_pretrained(final_path)

In [ ]:
prompt = "K. ouvrit la porte. "

for i in range(10):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=1000,
            temperature=0.8,
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.1
        )
    
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"== generation {i} ==")
    print(generated)
    print("\n")